In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.18.1


In [3]:
import features #functions defining word features
import data_parser #function(s) to load train and test data from .txt files
import cPickle as pickle
import pprint

freq=features.frequencies('train.txt') #returns a dictionary of word frequencies in the file

train_sents=data_parser.load('train.txt')
test_sents=data_parser.load('test.txt')
#sents is a list of lists, each list corresponding to a sentence in the 'train.txt'
#The list has the format (word,category,label) for each word in sentence, each label corresponding to
#the entity of the word. For current training datasets, these are-
#(Date-Date, Num-Number of tickets, Dest-Destination, Src-Source Location)
#category is retreived from the Hindi WordNet database,return N for noun,
#V for verb,AV for adverb and AJ for adjective. Return X if not found in the database 
print len(train_sents)
for i in train_sents:
    for x in i:
        print(x[0])

        

27
मैं
भोपाल
जाना
चाहता
हूं
मुझे
कल
के
लिए
मुंबई
के
दो
टिकट
दो
क्या
कल
चेन्नई
से
पांडिचेरी
तक
कोई
उड़ानें
उपलब्ध
हैं
मैं
दिल्ली
जाना
चाहता
हूँ
मुझे
दिल्ली
से
औरंगाबाद
की
कल
के
लिए
दो
टिकट
बुक
करें
मैं
इंग्लैंड
जाना
चाहता
हूं
अगले
महीने
अमेरिका
की
टिकट
कीमतों
को
दिखाएं
मैं
110702
को
मुंबई
जाना
चाहता
हूं
मैं
कल

गुवाहाटी
में
जाना
चाहता
हूं
मुझे
परसों
इंदौर
में
होना
चाहिए
मुझे
न्यूयॉर्क
जाने
की
जरूरत
है
मुझे
गोवा
ले
जाओ
मैं
परसों
नागपुर
जाना
चाहता
हूं
कल
मुंबई
से
श्रीनगर
के
लिए
4
टिकट
बुक
करें।
मैं
कल
दिल्ली
जाना
चाहता
हूं
मुझे
1
तारीख
की
इंदौर
से
चेन्नई
के
लिए
4
टिकटों
की
बुकिंग
करें
मैं
बेंगलुरु
जाना
चाहता
हूं
मुझे
अगले
सप्ताह
अमेरिका
में
होना
चाहिए
मुझे
इंग्लैंड
जाने
की
ज़रूरत
है
कल
दिल्ली
के
लिए
एक
टिकट
बुक
करें
जयपुर
से
लखनऊ
तक
22
वीं
की
कोई
बस
है
दिल्ली
की
कल
के
लिए
एक
टिकट
बुक
करें
क्या
कल
के
लिए
मुंबई
में
कोई
टिकट
उपलब्ध
है
मुझे
बॉम्बे
ले
आओ
मुझे
सिडनी
जाना
है
कल
लखनऊ
की
कोई
भी
उड़ानें
मैं
कल
मुंबई
जाना
चाहता
हूं


In [7]:
def word2features(sent, i):
    word = sent[i][0]
    category = sent[i][1]
    if(word not in freq): freq[word]=0
    feat = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % features.isdigit(word),
        'category=' + str(category),
        'freq='+ str(freq[word]),
    ]
    if i > 0:
        word1 = sent[i-1][0]
        if(word1 not in freq): freq[word1]=0
        category1 = sent[i-1][1]
        feat.extend([
            '-1:word=' + word,
            '-1:word.isdigit=%s' % features.isdigit(word1),
            '-1:category=' + str(category1),
            '-1:freq='+ str(freq[word1]),
        ])
    else:
        feat.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        if(word1 not in freq): freq[word1]=0
        category1 = sent[i+1][1]
        feat.extend([
            '+1:word=' + word,
            '+1:word.isdigit=%s' % features.isdigit(word1),
            '+1:category=' + str(category1),
            '+1:freq='+ str(freq[word1]),
        ])
    else:
        feat.append('EOS')
                
    return feat


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for word, category, label in sent]

def sent2tokens(sent):
    return [word for word,postag,label in sent]

In [8]:
sent2features(train_sents[0])[0]

['bias',
 'word=\xe0\xa4\xae\xe0\xa5\x88\xe0\xa4\x82',
 'word.isdigit=False',
 'category=X',
 'freq=9',
 'BOS',
 '+1:word=\xe0\xa4\xae\xe0\xa5\x88\xe0\xa4\x82',
 '+1:word.isdigit=False',
 '+1:category=X',
 '+1:freq=1']

In [9]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]
print y_train[3]

['0', 'D', '0', '0', '0']


In [10]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)


In [11]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})



In [12]:
trainer.params() #List set of possible params


['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [13]:
%%time
trainer.train('hindiNER.crfsuite') #train and save model to file 'hindiNER.crfsuite'

CPU times: user 52 ms, sys: 8 ms, total: 60 ms
Wall time: 47.6 ms


In [14]:
tagger = pycrfsuite.Tagger()
tagger.open('hindiNER.crfsuite')

In [17]:
for i in test_sents:
    example_sent = i#test_sents[0]
    print(' '.join(sent2tokens(example_sent)))
    print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
    print("Correct:  ", ' '.join(sent2labels(example_sent)))

कैनबरा के लिए टिकट बुक करें
('Predicted:', '0 0 0 0 0 0')
('Correct:  ', 'D 0 0 0 0 0')
मुंबई से गुवाहाटी तक सबसे सस्ता किराया क्या है
('Predicted:', 'S 0 D 0 0 0 0 S 0')
('Correct:  ', 'S 0 D 0 0 0 0 0 0')
दिल्ली के लिए टिकटों की जांच करें
('Predicted:', 'D 0 0 0 0 0 0')
('Correct:  ', 'D 0 0 0 0 0 0')
मुझे सिडनी ले जाएं
('Predicted:', '0 0 0 0')
('Correct:  ', '0 D 0 0')
अगले महीने लंदन के टिकटों की कीमत दिखाएं
('Predicted:', '0 0 0 0 0 0 0 0')
('Correct:  ', '0 0 D 0 0 0 0 0')
चेन्नई के लिए टिकट बुक करें
('Predicted:', '0 0 0 0 0 0')
('Correct:  ', 'D 0 0 0 0 0')
मुझे अगले महीने मुंबई के टिकटों की कीमत दिखाएं
('Predicted:', '0 0 0 D 0 0 0 0 0')
('Correct:  ', '0 0 0 D 0 0 0 0 0')


In [18]:
y_pred = [tagger.tag(xseq) for xseq in X_test]

In [19]:
#Evaluate model performance

Let us check what the classifier learnt

In [20]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
D      -> 0       1.725437
T      -> D       0.882015
0      -> D       0.116878
S      -> 0       0.082172
0      -> T       0.070536
T      -> S       0.032156
T      -> 0       -0.064591
0      -> 0       -0.156190
0      -> S       -0.456525

Top unlikely transitions:
D      -> 0       1.725437
T      -> D       0.882015
0      -> D       0.116878
S      -> 0       0.082172
0      -> T       0.070536
T      -> S       0.032156
T      -> 0       -0.064591
0      -> 0       -0.156190
0      -> S       -0.456525


In [21]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
3.320278 S      +1:freq=5
2.665667 N      -1:freq=7
1.944542 D      +1:freq=10
1.898065 T      word=कल
1.898065 T      freq=11
1.898065 T      +1:word=कल
1.883103 0      freq=9
1.828114 N      +1:freq=7
1.512897 D      -1:freq=5
1.364915 0      bias
1.364915 0      category=X
1.334110 0      -1:freq=1
1.273273 T      -1:freq=9
1.206931 0      word.isdigit=False
1.165126 0      +1:freq=1
1.008999 0      -1:freq=4
0.994211 0      freq=4
0.945364 0      freq=10
0.875313 0      EOS
0.865102 0      freq=8

Top negative:
0.504537 D      +1:freq=8
0.266940 D      +1:freq=9
0.246206 D      -1:word=अमेरिका
0.246206 D      word=अमेरिका
0.246206 D      +1:word=अमेरिका
0.155730 0      -1:category=X
0.046218 D      word=मुंबई
0.046218 D      -1:word=मुंबई
0.046218 D      +1:word=मुंबई
0.007354 0      -1:word=से
0.007354 0      word=से
0.007354 0      +1:word=से
0.003408 D      word=इंग्लैंड
0.003408 D      -1:word=इंग्लैंड
0.003408 D      +1:word=इंग्लैंड
-0.115082 D      -1:freq=9
-0